In [ ]:
## AGENDA
## - Page Rank

In [ ]:
#from py2neo import Graph
#graph = Graph("bolt://neo4j.sktai.io:30074", auth=("neo4j","!neo4j00"))

In [ ]:
# https://github.com/neo4j-contrib/neo4j-apoc-procedures/releases/tag/3.5.0.1
#mv ./apoc-3.5.0.1-all.jar $Neo4J_HOME/plugins

In [1]:
from neo4j import GraphDatabase
class Neo4jConnection:
    
    def __init__(self, uri, user, pwd):
        self.__uri = uri
        self.__user = user
        self.__pwd = pwd
        self.__driver = None
        try:
            self.__driver = GraphDatabase.driver(self.__uri, auth=(self.__user, self.__pwd))
        except Exception as e:
            print("Failed to create the driver:", e)
        
    def close(self):
        if self.__driver is not None:
            self.__driver.close()
        
    def query(self, query, db=None):
        assert self.__driver is not None, "Driver not initialized!"
        session = None
        response = None
        try: 
            session = self.__driver.session(database=db) if db is not None else self.__driver.session() 
            response = list(session.run(query))
        except Exception as e:
            print("Query failed:", e)
        finally: 
            if session is not None:
                session.close()
        return response

In [2]:
#gph_conn = Neo4jConnection(uri="bolt://neo4j.sktai.io:30074", user="neo4j", pwd="!neo4j00")
#gph_conn = Neo4jConnection(uri="bolt://localhost:7687", user="neo4j", pwd="!neo4j00")
gph_conn = Neo4jConnection(uri="bolt://127.0.0.1:7687", user="neo4j", pwd="!neo4j00")

In [3]:
gph_conn.query("""
               CALL db.schema();
               """)

[<Record nodes=[<Node id=-4 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>] relationships=[<Relationship id=-4 nodes=(<Node id=-4 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>, <Node id=-4 labels=frozenset({'Node'}) properties={'indexes': [], 'name': 'Node', 'constraints': []}>) type='LINKED_TO' properties={}>]>]

In [4]:
gph_conn.query("""
          MATCH (n)
          DETACH DELETE n
          """)

[]

In [ ]:
## SET DATA

In [5]:
gph_conn.query("""
               MERGE (A:Node {name: "A"})
               MERGE (B:Node {name: "B"})
               MERGE (C:Node {name: "C"})
               MERGE (D:Node {name: "D"})
               MERGE (E:Node {name: "E"})
               MERGE (F:Node {name: "F"})
               MERGE (G:Node {name: "G"})
               MERGE (Y:Node {name: "Y"})
               MERGE (Z:Node {name: "Z"})

               MERGE (A)-[:LINKED_TO]->(B)
               MERGE (B)-[:LINKED_TO]->(C)
               MERGE (C)-[:LINKED_TO]->(A)
               MERGE (D)-[:LINKED_TO]->(C)
               MERGE (D)-[:LINKED_TO]->(E)
               MERGE (E)-[:LINKED_TO]->(D)
               MERGE (E)-[:LINKED_TO]->(G)
               MERGE (F)-[:LINKED_TO]->(E)
               MERGE (G)-[:LINKED_TO]->(D)
               MERGE (G)-[:LINKED_TO]->(F)
               MERGE (Y)-[:LINKED_TO]->(Z)
               MERGE (Z)-[:LINKED_TO]->(Y)
               """)

[]

In [ ]:
## via GDS

In [7]:
gph_conn.query("""
               CALL gds.graph.drop("graph")
               """)
gph_conn.query("""
               CALL gds.graph.create("graph", "Node", "LINKED_TO")
               """)

Query failed: {code: Neo.ClientError.Procedure.ProcedureCallFailed} {message: Failed to invoke procedure `gds.graph.drop`: Caused by: java.lang.IllegalArgumentException: Graph with name `graph` does not exist and can't be removed.}


[<Record graphName='graph' nodeProjection={'Node': {'properties': {}, 'label': 'Node'}} relationshipProjection={'LINKED_TO': {'orientation': 'NATURAL', 'aggregation': 'DEFAULT', 'type': 'LINKED_TO', 'properties': {}}} nodeCount=9 relationshipCount=12 createMillis=10>]

In [9]:
gph_conn.query("""
               CALL gds.wcc.stream("graph")
               YIELD nodeId, componentId 
               RETURN gds.util.asNode(nodeId).name as nodeName, componentId 
               ORDER BY componentId
               """)

[<Record nodeName='Y' componentId=0>,
 <Record nodeName='Z' componentId=0>,
 <Record nodeName='A' componentId=2>,
 <Record nodeName='B' componentId=2>,
 <Record nodeName='C' componentId=2>,
 <Record nodeName='D' componentId=2>,
 <Record nodeName='E' componentId=2>,
 <Record nodeName='F' componentId=2>,
 <Record nodeName='G' componentId=2>]

In [11]:
gph_conn.query("""
               CALL gds.alpha.scc.stream("graph")
               YIELD nodeId, componentId 
               RETURN gds.util.asNode(nodeId).name as nodeName, componentId 
               ORDER BY componentId
               """)

[<Record nodeName='Y' componentId=0>,
 <Record nodeName='Z' componentId=0>,
 <Record nodeName='A' componentId=2>,
 <Record nodeName='B' componentId=2>,
 <Record nodeName='C' componentId=2>,
 <Record nodeName='D' componentId=5>,
 <Record nodeName='E' componentId=5>,
 <Record nodeName='F' componentId=5>,
 <Record nodeName='G' componentId=5>]

In [13]:
gph_conn.query("""
               CALL gds.wcc.write("graph", {writeProperty:'wcc'})
               """)

[<Record nodePropertiesWritten=9 createMillis=0 computeMillis=0 writeMillis=13 postProcessingMillis=2 componentCount=2 componentDistribution={'p99': 7, 'min': 2, 'max': 7, 'mean': 4.5, 'p90': 7, 'p50': 2, 'p999': 7, 'p95': 7, 'p75': 7} configuration={'writeConcurrency': 4, 'consecutiveIds': False, 'seedProperty': None, 'writeProperty': 'wcc', 'threshold': 0.0, 'relationshipWeightProperty': None, 'nodeLabels': ['*'], 'sudo': False, 'relationshipTypes': ['*'], 'concurrency': 4}>]

In [14]:
gph_conn.query("""
               MATCH (n:Node) 
               RETURN n.name as nodeName, n.wcc
               """)

[<Record nodeName='Y' n.wcc=0>,
 <Record nodeName='Z' n.wcc=0>,
 <Record nodeName='A' n.wcc=2>,
 <Record nodeName='B' n.wcc=2>,
 <Record nodeName='C' n.wcc=2>,
 <Record nodeName='D' n.wcc=2>,
 <Record nodeName='E' n.wcc=2>,
 <Record nodeName='F' n.wcc=2>,
 <Record nodeName='G' n.wcc=2>]